In [6]:
%load_ext autoreload
%autoreload 2

import jax
import jax.numpy as jnp
import optax
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

from basin_volume import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def make_kl_fn(probs_p, apply_fn, x, l2_reg):
    def kl_fn(a, b):
        params_q = a + b
        logits_q = apply_fn(params_q, x)
        logprobs_q = jax.nn.log_softmax(logits_q)
        kl_term = optax.kl_divergence(logprobs_q, probs_p).mean()
        l2_term = 1/2 * l2_reg * jnp.sum(b**2)
        return kl_term + l2_term
    return kl_fn

def make_kl_fn_params(params_p, apply_fn, x, *, l2_reg):
    logits_p = apply_fn(params_p, x)
    probs_p = jax.nn.softmax(logits_p)
    return make_kl_fn(probs_p, apply_fn, x, l2_reg=l2_reg)

# Tokenizer

In [2]:

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-14m")

In [3]:
len(tokenizer.vocab)

50277

# Model

In [7]:

model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-14m")

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [23]:
# for name, p in model.named_parameters():
#     print(name, p.shape)
# print()

# sort by number of parameters and print
sorted_params = sorted(model.named_parameters(), key=lambda x: np.prod(x[1].shape))
# compute total number of parameters
total_params = sum(np.prod(p.shape) for name, p in sorted_params)
print(f"Total number of parameters: {total_params}")

# compute total embedding parameters
embedding_params = sum(np.prod(p.shape) for name, p in sorted_params if "embed" in name)
print(f"Total number of embedding parameters: {embedding_params}")
print(f"Total number of non-embedding parameters: {total_params - embedding_params}")

for name, p in reversed(sorted_params):
    print(f"{np.prod(p.shape) / total_params:.2%}", p.shape, name)

Total number of parameters: 14067712
Total number of embedding parameters: 12877824
Total number of non-embedding parameters: 1189888
45.77% torch.Size([50304, 128]) embed_out.weight
45.77% torch.Size([50304, 128]) gpt_neox.embed_in.weight
0.47% torch.Size([128, 512]) gpt_neox.layers.5.mlp.dense_4h_to_h.weight
0.47% torch.Size([512, 128]) gpt_neox.layers.5.mlp.dense_h_to_4h.weight
0.47% torch.Size([128, 512]) gpt_neox.layers.4.mlp.dense_4h_to_h.weight
0.47% torch.Size([512, 128]) gpt_neox.layers.4.mlp.dense_h_to_4h.weight
0.47% torch.Size([128, 512]) gpt_neox.layers.3.mlp.dense_4h_to_h.weight
0.47% torch.Size([512, 128]) gpt_neox.layers.3.mlp.dense_h_to_4h.weight
0.47% torch.Size([128, 512]) gpt_neox.layers.2.mlp.dense_4h_to_h.weight
0.47% torch.Size([512, 128]) gpt_neox.layers.2.mlp.dense_h_to_4h.weight
0.47% torch.Size([128, 512]) gpt_neox.layers.1.mlp.dense_4h_to_h.weight
0.47% torch.Size([512, 128]) gpt_neox.layers.1.mlp.dense_h_to_4h.weight
0.47% torch.Size([128, 512]) gpt_neox.la